# MNIST-fashion으로 다중 분류하기

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
import pandas as pd
from sklearn.datasets import fetch_openml

mnist = fetch_openml("Fashion-MNIST", as_frame = False)
# MNIST 데이터는 이미지라서 데이터 프레임으로 할 필요 X

C:\Users\kdp\anaconda3\envs\Torch_PY38\lib\site-packages\sklearn\datasets\_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [38]:
data = mnist["data"] / 255 # 전처리
target = mnist["target"]

# Deep Learning?

In [39]:
data[:60000,:].dtype

dtype('float64')

In [40]:
target[:60000]

array([9, 0, 0, ..., 3, 0, 5], dtype=object)

In [41]:
target[:60000] = target[:60000].astype(int)

In [42]:
train_X = torch.tensor(data[:60000,:].astype("float32"))
test_X = torch.tensor(data[60000:,:].astype("float32"))
train_y = torch.tensor(target[:60000].astype("int64"))
test_y = torch.tensor(target[60000:].astype("int64"))

In [43]:
print(train_X.shape, train_X.type())
print(train_y.shape, train_y.type())
print(test_X.shape, test_X.type())
print(test_y.shape, test_y.type())

torch.Size([60000, 784]) torch.FloatTensor
torch.Size([60000]) torch.LongTensor
torch.Size([10000, 784]) torch.FloatTensor
torch.Size([10000]) torch.LongTensor


In [44]:
train_y = train_y.unsqueeze(-1)
test_y = test_y.unsqueeze(-1)

In [45]:
# 가중치와 편향 선언
# W = torch.zeros((784, 1), requires_grad=True) # 가중치는 feature개수만큼 
# b = torch.zeros(1, requires_grad=True)

In [27]:
# 모델 설계
# Sequential : 순서대로 진행되어야 하는 모듈을 묶음으로 관리 -> 파이프라인같은 건감...?ㅇㅅㅇ

model = nn.Sequential(
   nn.Linear(784, 10), # input_dim = 28*28, output_dim = 1
   nn.Softmax(dim=1)     # 다중은 softmax
) 

# 경사하강법으로 업데이트 해야하는 W, b는 모델 인스턴스에서 가지고 있음
optimizer = optim.SGD(model.parameters(), lr=1) # 

In [28]:
y_one_hot = torch.zeros(60000,10)
y_one_hot.scatter_(1, train_y, 1)

tensor([[0., 0., 0.,  ..., 0., 0., 1.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [31]:
# 학습

def training():
    nb_epochs = 100
    for epoch in range(nb_epochs+1):
        
        hypothesis = model(train_X) # y_predict 
        print(hypothesis.shape)
        # print(hypothesis.shape)
        # 다중 -> y_one_hot encoding

        cost = F.cross_entropy(hypothesis, y_one_hot)
        # (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
        # F.binary_cross_entropy(hypothesis, train_y)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        # 20번마다 로그 출력
        if epoch % 10 == 0:
            prediction = hypothesis >= torch.FloatTensor([0.5])     # 예측값이 0.5를 넘으면 True로 간주
            correct_prediction = prediction.float() == y_one_hot      # 실제값과 일치하는 경우만 True로 간주
            accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
 
            print(f'Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f} ') # Accuracy {accuracy * 100:2.2f}%

In [32]:
training()

torch.Size([60000, 10])
Epoch    0/100 Cost: 1.742513 
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([60000, 10])
Epoch   10/100 Cost: 1.737213 
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([60000, 10])
Epoch   20/100 Cost: 1.732727 
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([60000, 10])
Epoch   30/100 Cost: 1.728848 
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([

In [15]:
print(list(model.parameters()))

[Parameter containing:
tensor([[ 0.0184, -0.0146, -0.0139,  ..., -0.0471, -0.0198,  0.0254],
        [ 0.0103,  0.0006, -0.0238,  ..., -0.0177,  0.0064,  0.0339],
        [-0.0183,  0.0262, -0.0139,  ...,  0.0072,  0.0026, -0.0122],
        ...,
        [-0.0202,  0.0322, -0.0332,  ..., -0.0092, -0.0339,  0.0300],
        [-0.0172,  0.0002,  0.0303,  ...,  0.0219, -0.0294,  0.0021],
        [ 0.0233,  0.0181,  0.0270,  ...,  0.0070,  0.0089, -0.0266]],
       requires_grad=True), Parameter containing:
tensor([ 0.0569,  0.0309,  0.0142, -0.0071, -0.2064,  0.4995, -0.0209,  0.0517,
        -0.1417, -0.2126], requires_grad=True)]


# ML

In [33]:
train_X = data[:60000,:]
test_X = data[60000:,:]
train_y = target[:60000].astype("int64")
test_y = target[60000:].astype("int64")

In [34]:
# test train 분리는 이미 위에서 완료
# 다중 분류!! sklearn은 default ovr이라서 따로 뭐 할 거X
from sklearn.linear_model import LogisticRegression 
LR_model = LogisticRegression(max_iter=2000)

LR_model.fit(train_X, train_y)

LogisticRegression(max_iter=2000)

In [35]:
LR_model.score(train_X, train_y)

0.8810666666666667

In [36]:
LR_model.score(test_X, test_y)

0.8442

In [37]:
from sklearn.metrics import classification_report

y_pred = LR_model.predict(test_X)
print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.81      0.80      1000
           1       0.97      0.96      0.96      1000
           2       0.73      0.74      0.73      1000
           3       0.83      0.86      0.85      1000
           4       0.74      0.76      0.75      1000
           5       0.94      0.92      0.93      1000
           6       0.63      0.57      0.60      1000
           7       0.91      0.94      0.92      1000
           8       0.93      0.94      0.93      1000
           9       0.95      0.95      0.95      1000

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.84      0.84      0.84     10000
